In [3]:
!pip install -q chromadb langchain langchain_community sentence_transformers cohere

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\VISHAL MAURYA\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\kubernetes\\client\\models\\v1_custom_resource_definition.py'
Consider using the `--user` option or check the permissions.



  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached mmh3-4.1.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached pyproject_hooks-1.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached httptools-0.6.1-cp311-cp311-win_amd64.whl.metadata (3.7 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached pyreadline3-3.4.1-py3-none-any.whl.metadata (2.0 kB)
   ---------------------------------------- 0.0/584.3 kB ? eta -:--:--
   --------------------------------------- 584.3/584.3 kB 10.4 MB/s eta 0:00:00
Using cached build-1.2.1-py3-none-a

### Required Modules

In [1]:
from tqdm.autonotebook import tqdm, trange

C:\Users\VISHAL MAURYA\AppData\Local\Temp\ipykernel_12004\4256873726.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
import os
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_transformers import (
    EmbeddingsRedundantFilter,
    EmbeddingsClusteringFilter,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain.docstore.document import Document
from langchain_community.llms import Ollama
import time
from langchain.prompts import PromptTemplate
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

### Data Pre-processing

In [4]:
pdf_paths1 = ["data/GB 25991-2010 Automotive Headlamps with LED Light Sources andor LED Modules (1) (2).pdf"]
query1 = "The color of the light emitted by LED headlamps shall be?"
query2 = "The LED module of headlamp shall be operated for how many hours before cooled down to ambient temperature before starting the tests?"
query3 = "Tell me about Test of stability of photometric performance."
query4 = "What are environmental conditions for test for change in vertical position of the cut-off line under the influence ofheat?"
query5 = "For headlamp with the outside lens in glass, a mixture to be applied to the headlamp lens shall consist of which elements"
pdf_paths2 = ["Chinese Documents/GB 4785-2019 Lighting installation -EN.pdf"]

In [44]:
def get_documents(pdf_paths):
    documents = []
    for path in pdf_paths:
        loader = PyMuPDFLoader(path)
        list_of_documents = loader.load()
        combined_text = "\n".join(document.page_content for document in list_of_documents)
        document = Document(page_content=combined_text, metadata={"source": path})
        documents.append(document)
    return documents

In [45]:
documents1 = get_documents(pdf_paths1)
documents2 = get_documents(pdf_paths2)

### Embedding Model

In [7]:
s = time.time()
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceBgeEmbeddings(model_name=model_name,
                                            model_kwargs=model_kwargs,
                                            encode_kwargs=encode_kwargs)
print("Embedding Model Loaded in", time.time() - s, "seconds")

C:\Users\VISHAL MAURYA\Desktop\Document Comparision\pdr_venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding Model Loaded in 31.029078483581543 seconds


### Parent document retriever

In [46]:
def get_retriever(pdf_paths):
    documents = get_documents(pdf_paths)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=100)
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
    
    vectorstore = Chroma(collection_name="full_documents", embedding_function=embedding_model)
    store = InMemoryStore()
    retriever = ParentDocumentRetriever(vectorstore=vectorstore,
                                        docstore=store,
                                        child_splitter=child_splitter,
                                        parent_splitter=parent_splitter)
    
    s = time.time()
    retriever.add_documents(documents, ids=None)
    print("Retriever created in", (time.time()-s)/60, "minutes")

    return retriever

In [47]:
def get_vectorstore(pdf_paths):
    documents = get_documents(pdf_paths)
    vectorstore = Chroma(collection_name="full_documents", 
                         embedding_function=embedding_model)
    return vectorstore

In [10]:
retriever1 = get_retriever(pdf_paths1)

Retriever created in 6.049975363413493 minutes


In [11]:
retriever2 = get_retriever(pdf_paths2)

Retriever created in 27.045053990681968 minutes


In [48]:
vectorstore1 = get_vectorstore(pdf_paths1)
vectorstore2 = get_vectorstore(pdf_paths2)

In [49]:
sub_docs = vectorstore1.similarity_search(query1, k = 6)

In [50]:
retrieved_docs = retriever1.get_relevant_documents(query1)
# retrieved_docs

In [51]:
def get_context(query, retriever):
    context = ""
    retrieved_docs = retriever.get_relevant_documents(query)
    for i in range(len(retrieved_docs)):
      # print("----------------------------------------- parent",i+1)
        # print(retrieved_docs[i].page_content)
        context += retrieved_docs[i].page_content
    return context

# Text Generation

### LLM and prompt

In [24]:
model = Ollama(model="llama3")

In [52]:
def create_prompt(query, context):
    template = """
    Answer the question based on the context below. 
    Don't start the response from "Based on the context provided, the answer to the question is:" or something like that, just given the answer.
    If you can't answer the question, reply "I don't know". 
    This is very important for my life, be accurate and precised.
    
    CONTEXT: {context}
    
    QUESTION: {question}
    """
    prompttemplate = PromptTemplate.from_template(template)
    prompt = prompttemplate.format(context = context, question = query)
    return prompt

### For pdf 1

In [26]:
queries1 = [query1, query2, query3, query4, query5] # For Automotive Headlamps

In [32]:
for query in queries1:
    st = time.time()
    print("QUERY:", query)
    context = get_context(query, retriever1)
    prompt = create_prompt(query, context)
    print("RESPONSE:", model.invoke(prompt))
    print("GENERATION TIME:", (time.time() - st)/60, "minutes")
    print("")

QUERY: The color of the light emitted by LED headlamps shall be?
RESPONSE: White
GENERATION TIME: 5.141609108448028 minutes

QUERY: The LED module of headlamp shall be operated for how many hours before cooled down to ambient temperature before starting the tests?
RESPONSE: According to the context provided, the LED module of the headlamp shall be operated for 15 hours before being cooled down to ambient temperature before starting the tests.
GENERATION TIME: 4.4921447277069095 minutes

QUERY: Tell me about Test of stability of photometric performance.
RESPONSE: The Test of Stability of Photometric Performance is a test conducted in accordance with GB/T 7922 "Methods of Measuring the Color of Light Source" to evaluate the stability of the photometric performance of LED headlamps. The test involves measuring the direct beam of light from the headlamp after correct aiming and stabilization, and conducting colorimetric checks using a spectrophotometer to determine the color temperature an

### For pdf 2

In [33]:
queries2 = ["Whats the difference between Grouped and Combined lamps?", "Can  dipped-beam headlamp and main-beam headlamp for front lighting system?", "what is color of End Outline marker lamp?", "Can yellow lamp used as front fog lamp?", "Can red color light placed in the front of the vechiile?", "Can white light can be placed at the back of the vechile?", "What are 1,1,a,1b,2a,2b,5,6 in direction indicator lamps?", "is cornering lamp mandatory?", "does reflective tape come under light and light signalling?", "standard weight of a person for testing?","can dipped beam uses as a main beam?", "what are the light functions to be kept rear of the vechile?"]

In [36]:
for query in queries2:
    st = time.time()
    print("QUERY:", query)
    context = get_context(query, retriever2)
    prompt = create_prompt(query, context)
    print("RESPONSE:", model.invoke(prompt))
    print("GENERATION TIME:", (time.time() - st)/60, "minutes")
    print("")

QUERY: Whats the difference between Grouped and Combined lamps?
RESPONSE: Grouped and combined lamps differ in their design and functionality. Here is the difference between them based on the provided context:

Grouped lamps:

* Have separate apparent surfaces but share a common lamp body
* Are positioned in the same direction as the reference axis (the axis of the lamp)
* Do not overlap each other based on their light emitting surfaces

Combined lamps:

* Have separate apparent surfaces but share a common light source and lamp body
* Are positioned in the same direction as the reference axis (the axis of the lamp)
* Do not overlap each other based on their light emitting surfaces

In summary, grouped lamps have multiple apparent surfaces that are part of the same lamp body, while combined lamps have a single light source and lamp body with multiple apparent surfaces.
GENERATION TIME: 8.660244544347128 minutes

QUERY: Can  dipped-beam headlamp and main-beam headlamp for front lighting 

### Prompting whole pdf 2 to llama2

In [54]:
documents = get_documents(pdf_paths2)

In [55]:
queries2 = ["Whats the difference between Grouped and Combined lamps?", "Can dipped-beam headlamp and main-beam headlamp for front lighting system?", "what is color of End Outline marker lamp?", "Can yellow lamp used as front fog lamp?", "Can red color light placed in the front of the vechiile?", "Can white light can be placed at the back of the vechile?", "What are 1,1,a,1b,2a,2b,5,6 in direction indicator lamps?", "is cornering lamp mandatory?", "does reflective tape come under light and light signalling?", "standard weight of a person for testing?","can dipped beam uses as a main beam?", "what are the light functions to be kept rear of the vechile?"]

In [56]:
for query in queries2:
    st = time.time()
    print("QUERY:", query)
    context = documents[0].page_content
    prompt = create_prompt(query, context)
    print("RESPONSE:", model.invoke(prompt))
    print("GENERATION TIME:", (time.time() - st)/60, "minutes")
    print("")

QUERY: Whats the difference between Grouped and Combined lamps?
RESPONSE: Thank you for your question! In the context of vehicle lighting, "Grouped" and "Combined" lamps refer to different types of lamp configurations. Here's a brief explanation:

Grouped Lights: In this configuration, multiple lamps are mounted on the same axis and are aimed in the same direction. The lamps are typically spaced at regular intervals along the vehicle's length, usually every 30-40 cm. Grouped lights are used to provide a wide beam of light that illuminates the road ahead without creating glare for other drivers.

Combined Lights: In this configuration, two or more lamps are mounted on different axes and aimed in different directions. The lamps are typically spaced at regular intervals along the vehicle's length, usually every 30-40 cm. Combined lights are used to provide a wider field of vision than grouped lights while minimizing glare for other drivers.

In summary, Grouped lights are mounted on the s

### Reranking

In [ ]:
import cohere
# w8CnnlzVol2aZEiirZNLUs0onAqXUUYBZCw2Oj7g
co = cohere.Client('sbF5OoltHNT3IHoIdu6aIi7W6WOiJD26i6x53v7D')

In [ ]:
def rerank(retrieved_docs, query):
    retrieved_texts = []
    for i in range(len(retrieved_docs)):
        retrieved_texts.append(retrieved_docs[i].page_content)
    ranking = co.rerank(query=query, documents=retrieved_texts, top_n=6, model='rerank-english-v2.0')
    for idx, r in enumerate(ranking.results):
        print(f"Document Rank: {idx + 1}, Document Index: {r.index}, Relevance Score: {r.relevance_score:.2f}")
        print(f"Source Document: {retrieved_docs[r.index].metadata}")
        print(f"Document: {retrieved_texts[r.index]}")
        print("-------------------------------------------------------------")

In [ ]:
rerank(retrieved_docs, query1)

### Comparing AEBS Documents

In [58]:
pdf_paths3 = ["Documents - Renamed/GB AEBS.pdf"]
pdf_paths4 = ["Documents - Renamed/UN AEBS.pdf"]

In [60]:
retriever3 = get_retriever(pdf_paths3)

Retriever created in 3.133275322119395 minutes


In [61]:
retriever4 = get_retriever(pdf_paths4)

Retriever created in 3.5497378667195636 minutes


In [67]:
def create_comparision_prompt(query, context1, context2):
    template = """
    Answer in three sections
    (1) ANSWER 1: tell answer form the context 1.
    (2) ANSWER 2: tell answer form the context 2.
    (3) COMPARISON: tell whether both answer are same or partially same or different. If they are paritially same, then what is same and what is different.
    If you can't answer the QUESTION from any context, reply "I don't know" for that context. 
    This is very important for my life, be very precised and accurate.

    QUESTION: {question}
    
    CONTEXT1: {context1}

    CONTEXT2: {context2}
    """
    prompttemplate = PromptTemplate.from_template(template)
    prompt = prompttemplate.format(context1 = context1,context2 = context2, question = query)
    return prompt

In [70]:
for query in ["The total speed reduction of the subject vehicle at the time of the collision with the stationary target shall be not less than how many kilometers per hour?"]:
    st = time.time()
    print("QUERY:", query)
    context3 = get_context(query, retriever3)
    context4 = get_context(query, retriever4)
    prompt = create_comparision_prompt(query, context3, context4)
    print("RESPONSE:", model.invoke(prompt))
    print("-------------------------------------------------- in:", (time.time() - st)/60, "minutes")
    print("")

QUERY: The total speed reduction of the subject vehicle at the time of the collision with the stationary target shall be not less than how many kilometers per hour?
RESPONSE: Answer in three sections:

Section 1: Answer from Context 1
The total speed reduction of the subject vehicle at the time of the collision with the stationary target shall be not less than 10km/h, as specified in Context 1.

Section 2: Answer from Context 2
The total speed reduction of the subject vehicle at the time of the collision with the stationary target shall be not less than the value specified in Table I, column D of Annex 3, as specified in Context 2.

Section 3: Comparison
Both answers are different. In Context 1, the speed reduction is specifically limited to 10km/h, while in Context 2, the speed reduction is specified to be not less than the value in Table I, column D of Annex 3. Therefore, the answer from Context 1 is different from the answer in Context 2.

Additionally, it's important to note that t

In [69]:
for query in ["When collision early warning signal shall be sent?"]:
    st = time.time()
    print("QUERY:", query)
    context3 = get_context(query, retriever3)
    context4 = get_context(query, retriever4)
    prompt = create_comparision_prompt(query, context3, context4)
    print("RESPONSE:", model.invoke(prompt))
    print("-------------------------------------------------- in:", (time.time() - st)/60, "minutes")
    print("")

QUERY: When collision early warning signal shall be sent?
RESPONSE: 
(1) Answer from Context 1:
When the AEBS (Automatic Emergency Braking System) detects the possibility of a collision with a forward vehicle in the same lane, which is traveling at a slower speed, has slowed to a halt or is stationary, it shall send a collision early warning signal. This signal shall be sent when the AEBS has detected the risk of collision and the timing of the warning is important to enable the driver to react timely and avoid nuisance warnings (4.2.2). The warning signal can be acoustic, haptic or optical, and if an optical signal is used, it may also flash to indicate a failure (4.2.3).

(2) Answer from Context 2:
In Context 2, the question is asked differently, focusing on the timing of the collision early warning signal. The answer is that the signal shall be sent when the AEBS detects the possibility of a collision with a forward vehicle in the same lane, which is traveling at a slower speed, has

In [71]:
query33 = "What AEBS should do in vehicle ignition?"
query = query33
st = time.time()
print("QUERY:", query)
context3 = get_context(query, retriever3)
context4 = get_context(query, retriever4)
prompt = create_comparision_prompt(query, context3, context4)
print("RESPONSE:", model.invoke(prompt))
print("-------------------------------------------------- generated in", (time.time() - st)/60, "minutes")

QUERY: What AEBS should do in vehicle ignition?
RESPONSE: Answer in three sections:

(1) ANSWER 1 (Context 1): The AEBS should deactivate when the ignition switch is turned to the "off" position. This is specified in the test report for vehicles equipped with a means to deactivate the AEBS function, as per 5.4.2.

(2) ANSWER 2 (Context 2): The vehicle manufacturer shall provide a list of positive actions to the technical service at the time of type approval, which includes deactivation of the AEBS function when the ignition switch is turned to the "off" position. This is specified in the vehicle manufacturer's guidelines for the AEBS system.

(3) COMPARISON: Both answer 1 and answer 2 indicate that the AEBS should deactivate when the ignition switch is turned to the "off" position. Therefore, the answers are partially the same. The difference lies in the context in which the answers were provided. Answer 1 was provided in the context of a test report for vehicles equipped with a means 